In [2]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

#Folders and Files

# Folders
repositoryFolder = "D:/Repositories/Global-COVID-Surveillance/data/"
localDownloadFolder = "C:/Users/janin/Downloads/"
demographicsFolder = repositoryFolder + "raw/demographics/"
configuredFolder = repositoryFolder + "configured/"
cleanedFolder = repositoryFolder + "cleaned/"
regionsFolder = repositoryFolder + "raw/regions/"
locationsFolder = repositoryFolder + "raw/locations/"

# Population Input Files
global_population_input_file = demographicsFolder + "Country Populations 2020.xlsx"
world_bank_populations_file = demographicsFolder + "Popular Indicators.xlsx"
us_codes = demographicsFolder + "US State Codes.xlsx"

# Population Output Files
canada_population_file = demographicsFolder + "Canada Population.xlsx"
us_population_file = demographicsFolder + "US Population.xlsx"
all_populations_file = cleanedFolder + "Populations_cleaned.xlsx"

def titleCase(words):
    if len(words) > 3:
        titlecased = ""
        wordsArray = words.lower().split(" ")
        for word in wordsArray:
            if len(titlecased) > 0 :
                titlecased = titlecased + " "
            if word == "and":
                titlecased = titlecased + "and"
            else:
                titlecased = titlecased + word.capitalize()
        return titlecased
    else:
        return words.upper()

def fixRegion(code):
    region_name = ""
    for region in census_regions:
        if region["number"] == code:
            region_name = region["name"]
            break
    if region_name == "":
        region_name = "Other"
        print(str(code) + " not found")
    return region_name

# CDC Standard age ranges 0-17, 18-29, 30-49, and 50-64
# CDC COVID Reporting Age Ranges https://www.cdc.gov/nchs/nvss/vsrr/covid_weekly/index.htm
def getAgeRange(age):
    age_range = ""
    if age == 0:
        age_range = "< 1"
    elif age == 999:
        age_range = "Total"
    elif age < 5:
        age_range = "1-4"
    elif age < 15:
        age_range = "5-14"
    elif age < 25:
        age_range = "15-24"
    elif age < 35:
        age_range = "25-34"
    elif age < 45:
        age_range = "35-44"
    elif age < 55:
        age_range = "45-54"
    elif age < 65:
        age_range = "55-64"
    elif age < 75:
        age_range = "65-74"
    elif age < 85:
        age_range = "75-84"
    elif age == 85:
        age_range = "85+"
    return age_range

def fixSex(code):
    sex = ""
    if code == 0:
        sex = "Population 2019"
    elif code == 1:
        sex = "Male"
    elif code == 2:
        sex = "Female"
    else:
        print(str(code) + " is not a sex")
    return sex

def us_date(x):
    month = x[5:7]
    day = x[8:11]
    year = x[0:4]
    conversion = month + "/" + day +"/"+ year
    return conversion

def removeDecimal(data):
    strData = str(data)
    decimalLocation = strData.find(".")
    if decimalLocation > -1:
        return strData[0:decimalLocation]
    else:
        return strData

def emptyNan(value):
    if (value == "nan"):
        return ""
    else:
        return value

def printColumns(df, label):
    print(label)
    print(df.columns)

def print_column_unique(column):
    print("Column Values:")
    values = column.sort_values(ascending = True).unique()
    print(values)
    return values

def print_column_missing(column, comparison):
    values = print_column_unique(column)
    print("Comparison:")
    print(comparison)
    missing_values = []
    for value in values:
        if not value in comparison:
            missing_values.append(value)
    if len(missing_values) > 0:
        print("Column values not in comparison:")
        print(missing_values)
    else:
        print("No missing values")
    missing_values = []
    for value in comparison:
        if not value in values:
            missing_values.append(value)
    if len(missing_values) > 0:
        print("Comparison values not in column:")
        print(missing_values)
    else:
        print("No missing values")
    return values

def division(a,b):
    if not (b == 0):
        return a/b
    else:
        return np.nan
    
# Countries and Regions

european_countries = [
    'Albania','Andorra','Austria','Belarus','Belgium','Bosnia & Herzegovina','Bulgaria',
    'Croatia','Czech Republic','Denmark','Estonia','Finland','France',
    'Germany','Greece','Greenland','Hungary','Iceland','Ireland','Isle of Man','Italy',
    'Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Moldova','Monaco','Montenegro',
    'Netherlands','Norway','Poland','Portugal','Romania',
    'San Marino','Serbia','Slovakia','Slovenia','Spain','Sweden','Switzerland',
    'Ukraine','United Kingdom','Vatican City'
]
carribean_countries = [
    "Antigua & Barbuda","Aruba","Bahamas","Barbados","Bermuda","British Virgin Islands",
    "Cayman Islands","Cuba","Curacao","Dominica","Dominican Republic","Grenada",
    "Haiti","Jamaica","Puerto Rico",
    "St. Barthelemy","St. Kitts & Nevis","St. Lucia","St. Vincent & Grenadines",
    "Sint Maarten","Trinidad & Tobago","Turks and Caicos Islands","United States Virgin Islands"
]
central_south_america_countries = [
    'Argentina','Belize','Bolivia','Brazil','Chile','Colombia','Costa Rica',
    'Ecuador','El Salvador','Guatemala','Guyana','Honduras',
    'Mexico','Nicaragua','Panama','Paraguay','Peru','Suriname','Uruguay','Venezuela'
]
latin_american_countries = carribean_countries + central_south_america_countries
sub_saharan_african_countries = [
    "Angola","Benin","Botswana","Burkina Faso","Burundi",
    "Cabo Verde","Cameroon","Central African Republic","Chad","Comoros","Côte d’Ivoire",
    "Democratic Republic of Congo","Equatorial Guinea","Eritrea","Ethiopia",
    "Gabon","Gambia","Ghana","Guinea","Guinea-Bissau","Kenya","Lesotho","Liberia",
    "Madagascar","Malawi","Mali","Mauritania","Mauritius","Mozambique",
    "Namibia","Niger","Nigeria","Republic of the Congo","Rwanda",
    "São Tomé and Príncipe","Senegal","Seychelles","Sierra Leone",
    "Somalia","South Africa","South Sudan","Sudan","Swaziland",
    "Tanzania","Togo","Uganda","Zambia","Zimbabwe"
]
south_asia_countries = [
    "Afghanistan","Bangladesh","Bhutan","India","Maldives","Nepal","Pakistan","Sri Lanka"
]
central_asian_countries = [
    'Armenia','Azerbaijan','Cyprus','Faeroe Islands','Georgia','Gibraltar','Kazakhstan','Kosovo','Kyrgyzstan',
    'North Macedonia','Russia','Tajikistan','Turkey','Turkmenistan','Uzbekistan'
]
east_asian_countries = [
    "Brunei","Cambodia","China","Indonesia","Japan","Laos","Malaysia","Mongolia","Myanmar","Niue","North Korea","Philippines",
    "Singapore","South Korea","Taiwan","Thailand","Timor","Vietnam"
]
pacific_countries = [
    "Australia","Cook Islands","Fiji","French Polynesia","Guam","Kiribati",
    "Marshall Islands","Micronesia","Nauru","New Caledonia","New Zealand",
    "Northern Mariana Islands","Palau","Papua New Guinea","Samoa","Solomon Islands","Tonga","Tuvalu","Vanuatu"
]
east_asia_and_pacific_countries = east_asian_countries + pacific_countries
middle_eastern_countries = [
    "Bahrain","Iran","Iraq","Israel","Jordan","Kuwait","Lebanon","Oman","Qatar",
    "Saudi Arabia","Syria","State of Palestine",
    "United Arab Emirates","Yemen"
]
north_african_countries = [
    "Algeria","Djibouti","Egypt","Libya","Morocco","Tunisia","Western Sahara"
]
middle_east_and_north_africa_countries = middle_eastern_countries + north_african_countries 
north_american_countries = ["Canada","United States"]
configured_country_lists = [
    european_countries,
    latin_american_countries,
    sub_saharan_african_countries,
    south_asia_countries,
    central_asian_countries,
    middle_east_and_north_africa_countries,
    east_asia_and_pacific_countries
]
configured_countries = []
for country_list in configured_country_lists:
    for country in country_list:
        configured_countries.append(country)
#print("Configured Countries")
configured_countries.sort()
#print(configured_countries)
#print(configured_countries)
all_countries = configured_countries +  north_american_countries
all_countries.sort()
#print("All Countries")
#print(all_countries)
#print("Configured Regions")
configured_regions = [
    'Central Asia',
    'East Asia and Pacific',
    'Europe',
    'Latin America',
    'Middle East and North Africa',
    'North America',
    'South Asia',
    'Sub-Saharan Africa'
]
#print(configured_regions)
#print("Country Conversions")
country_conversions = {
    "Antigua & Barbuda": ["Antigua and Barbuda"],
    "Bahamas": ["Bahamas, The"],
    "Bosnia & Herzegovina": ["Bosnia and Herzegovina"],
    "Brunei": ["Brunei Darussalam"],
    "Cabo Verde": ["Cape Verde"],
    "Côte d’Ivoire": ["Cote d'Ivoire","Cote dIvoire"],
    "Czech Republic": ["Czechia","Czech Republic (Czechia)"],
    "Democratic Republic of Congo": ["Congo - Kinshasa"],
    "Egypt": ["Egypt, Arab Rep."],
    "Faeroe Islands": ["Faroe Islands"],
    "Gambia": ["Gambia, The"],
    "Iran": ["Iran, Islamic Rep."],
    "Kyrgyzstan": ["Kyrgyz Republic"],
    "Laos": ["Lao PDR"],
    "Micronesia": ["Micronesia, Fed. Sts."],
    "Myanmar": ["Myanmar (Burma)","Burma"],
    "North Macedonia": ["Macedonia"],
    "Republic of the Congo": ["Congo - Brazzaville"],
    "Russia": ["Russian Federation"],
    "São Tomé and Príncipe": ["Sao Tome and Principe","Sao Tome & Príncipe","São Tomé & Príncipe"],
    "Sint Maarten": ["Sint Maarten (Dutch part)"],
    "Slovakia": ["Slovak Republic"],
    "St. Kitts & Nevis": ["Saint Kitts and Nevis"],
    "St. Lucia": ["Saint Lucia"],
    "St. Vincent & Grenadines": ["Saint Vincent and the Grenadines"],
    "Swaziland": ["Eswatini"],
    "Syria": ["Syrian Arab Republic"],
    "Timor": ["Timor-Leste"],
    "Trinidad & Tobago": ["Trinidad and Tobago"],
    "Vatican City": ["Holy See"],
    "Yemen": ["Yemen, Rep."],
    "" : ["nan"]
}
unincorporated_disputed_territories = [
    "American Samoa", "Anguilla","Caribbean Netherlands","Channel Islands","Curaçao",
    "Falkland Islands","French Guiana","Guadeloupe","Hong Kong"
]
territories = {
    "American Samoa":{"Region":"North America","Country":"United States"}, 
    "Anguilla":{"Region":"Europe","Country":"United Kingdom"},
    "Caribbean Netherlands":{"Region":"Europe","Country":"Netherlands"},
    "Channel Islands":{"Region":"Europe","Country":"Channel Islands"},
    "Curaçao":{"Region":"Europe","Country":"Netherlands"},
    "Falkland Islands":{"Region":"Europe","Country":"United Kingdom"},
    "French Guiana":{"Region":"Europe","Country":"France"},
    "Guadeloupe":{"Region":"Europe","Country":"France"},
    "Hong Kong":{"Region":"East Asia and Pacific","Country":"China"},
    "Macao":{"Region":"East Asia and Pacific","Country":"China"},
    "Martinique":{"Region":"Europe","Country":"France"},
    "Mayotte":{"Region":"Europe","Country":"France"},
    "Montserrat":{"Region":"Europe","Country":"United Kingdom"},
    "Réunion":{"Region":"Europe","Country":"France"},
    "St. Helena":{"Region":"Europe","Country":"United Kingdom"},
    "St. Martin":{"Region":"Europe","Country":"France"},
    "St. Pierre & Miquelon":{"Region":"Europe","Country":"France"},
    "Tokelau":{"Region":"East Asia and Pacific","Country":"New Zealand"},
    "Turks and Caicos":{"Region":"Europe","Country":"United Kingdom"},
    "U.S. Virgin Islands":{"Region":"North America","Country":"United States"},
    "Wallis & Futuna":{"Region":"Europe","Country":"France"}
}
#print(country_conversions)
#print("Countries by Region")
countries_by_region = {
    'Central Asia': central_asian_countries,
    'Europe': european_countries,
    'Latin America': latin_american_countries,
    'South Asia': south_asia_countries,
    'Sub-Saharan Africa': sub_saharan_african_countries,
    'Middle East and North Africa': middle_east_and_north_africa_countries,
    'East Asia and Pacific': east_asia_and_pacific_countries,
    'North America': north_american_countries
}
#print(countries_by_region)

def key_from_value(value, dictionary, default):
    return_value = default
    for key, values in dictionary.items():
        if value.strip() in values:
            return_value = key
            break
    return return_value.strip()

def region_from_country(country):
    return key_from_value(country, countries_by_region, "")

def fixCountry(value):
    return key_from_value(value, country_conversions, value)

def fixCountries(countries_column, configuredCountries):
    countries_conversion = countries_column.astype(str)
    countries_conversion = countries_conversion.apply(lambda x: fixCountry(x))
    print(conversions)
    countries = print_column_missing(countries_conversion,configuredCountries)
    return countries_conversion

def testConversion(title, test_array, conversion):
    print(title)
    no_conversions = []
    for value in test_array:
        return_value = ""
        if conversion == "country":
            return_value = fixCountry(value)
        elif conversion == "region":
            return_value = region_from_country(fixCountry(value))
        if return_value != value.strip():
            print(value.strip() + "," + return_value)
        if return_value == "":
            no_conversions.append(value)
    if len(no_conversions) > 0:
        print("Missing Conversions")
        print(no_conversions)
    print("")

In [155]:
print(world_bank_populations_file)
country_populations = pd.read_excel(world_bank_populations_file)
print(country_populations.columns)
country_populations.drop(columns=['Series Name', 'Series Code'],inplace=True)
country_populations.rename(columns={
    'Country Name':'Country',
    '2019 [YR2019]':'Population'
},inplace=True)
country_populations["Population"]
country_populations["Population"] = country_populations["Population"].astype(int)
country_populations["Population 100K"] = country_populations["Population"]/100000
country_populations.head()

D:/Repositories/Global-COVID-Surveillance/data/raw/demographics/Popular Indicators.xlsx
Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '2019 [YR2019]'],
      dtype='object')


ValueError: invalid literal for int() with base 10: '..'

In [64]:
kosovo_data = [["Country","Central Asia","Kosovo","","XK",1810366,18.10366]]
kosovo_columns = ["Level","Region","Country","State/Province","Abbreviation","Population","Population 100K"]
kosovo = pd.DataFrame(kosovo_data,columns=kosovo_columns)
kosovo.head()

,Level,Region,Country,State/Province,Abbreviation,Population,Population 100K
0,Country,Central Asia,Kosovo,,XK,1810366,18.10366


In [65]:
populationsFile = demographicsFolder + "Country Populations 2020.xlsx"
#print(populationsFile)
country_populations = pd.read_excel(populationsFile)
country_populations = country_populations.drop(columns=["Rank"])
country_populations.rename(
    columns = {
        'Data Source':'Population Data Source'
    }, 
    inplace = True)
country_populations["Population 100K"] = country_populations["Population"]/100000
conversions = {}
country_populations["Country"] = country_populations["Country"].astype(str)
country_populations["Country"] = country_populations["Country"].apply(lambda x: fixCountry(x))
country_populations["Region"] = country_populations["Country"].apply(lambda x: region_from_country(x))
country_populations["Level"] = country_populations["Region"].apply(lambda x: "Territory" if x == "" else "Country")
print(country_populations.columns)
def fixTerritoryRegion(territory):
    characteristics = territories[territory]
    return characteristics["Region"]
country_populations["Region"] = country_populations.apply(lambda x: fixTerritoryRegion(x["Country"]) if x["Region"] == "" else x["Region"],axis=1)
country_populations["State/Province"] = ""
print(conversions)
print(country_populations.columns)
country_populations_order = [
    'Level','Region','Country', 'State/Province',
    'Population', 'Population 100K', 
    'World Share (%)', 'Urban Population (%)', 
    'Annual Change (%)', 'Net Change', 'Migrants (net)', 
    'Density (P/Km²)', 'Land Area (Km²)', 
    'Fertility Rate', 'Median Age'
]
country_populations = pd.concat([country_populations,kosovo],sort=True)
country_populations = country_populations.sort_values(by=["Region","Country"])
country_populations = country_populations[country_populations_order]
country_populations.head(20)

Index(['Country', 'Country Link', 'Year', 'Population', 'Annual Change (%)',
       'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fertility Rate', 'Median Age', 'Urban Population (%)',
       'World Share (%)', 'Population Data Source', 'Population 100K',
       'Region', 'Level'],
      dtype='object')
{}
Index(['Country', 'Country Link', 'Year', 'Population', 'Annual Change (%)',
       'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fertility Rate', 'Median Age', 'Urban Population (%)',
       'World Share (%)', 'Population Data Source', 'Population 100K',
       'Region', 'Level', 'State/Province'],
      dtype='object')


,Level,Region,Country,State/Province,Population,Population 100K,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age
9,Country,Central Asia,Armenia,,2963243,29.63243,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0
13,Country,Central Asia,Azerbaijan,,10139177,101.39177,0.13,56.0,0.91,91459.0,1200.0,123.0,82658.0,2.1,32.0
52,Country,Central Asia,Cyprus,,1207359,12.07359,0.02,67.0,0.73,8784.0,5000.0,131.0,9240.0,1.3,37.0
67,Country,Central Asia,Faeroe Islands,,48863,0.48863,0.00,43.0,0.38,185.0,NaN,35.0,1396.0,NaN,NaN
76,Country,Central Asia,Georgia,,3989167,39.89167,0.05,58.0,-0.19,-7598.0,-10000.0,57.0,69490.0,2.1,38.0
79,Country,Central Asia,Gibraltar,,33691,0.33691,0.00,NaN,-0.03,-10.0,NaN,3369.0,10.0,NaN,NaN
106,Country,Central Asia,Kazakhstan,,18776707,187.76707,0.24,58.0,1.21,225280.0,-18000.0,7.0,2699700.0,2.8,31.0
0,Country,Central Asia,Kosovo,,1810366,18.10366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,Country,Central Asia,Kyrgyzstan,,6524195,65.24195,0.08,36.0,1.69,108345.0,-4000.0,34.0,191800.0,3.0,26.0
153,Country,Central Asia,North Macedonia,,2083374,20.83374,0.03,59.0,0.00,-85.0,-1000.0,83.0,25220.0,1.5,39.0


In [66]:
region_populations = country_populations.groupby(["Region"])["Population"].sum().reset_index()
region_populations["Region Population 100K"] = region_populations["Population"]/100000
region_populations.rename(columns={
    "Population":"Region Population"
},inplace=True)
region_populations["World"] = "World"
region_populations.head(10)

world_population = region_populations.groupby(["World"])["Region Population"].sum().reset_index()
world_population["World Population 100K"] = world_population["Region Population"]/100000
world_population.rename(columns={
    "Region Population":"World Population"
},inplace=True)
world_population.head(10)

region_populations = region_populations.merge(world_population,how="left",on="World").reset_index().drop(columns=["index","World"])
region_populations.head(15)

,Region,Region Population,Region Population 100K,World Population,World Population 100K
0,Central Asia,326887719,3268.87719,7796609105,77966.09105
1,East Asia and Pacific,2389320842,23893.20842,7796609105,77966.09105
2,Europe,602148888,6021.48888,7796609105,77966.09105
3,Latin America,652554939,6525.54939,7796609105,77966.09105
4,Middle East and North Africa,464362948,4643.62948,7796609105,77966.09105
5,North America,368904421,3689.04421,7796609105,77966.09105
6,South Asia,1856376663,18563.76663,7796609105,77966.09105
7,Sub-Saharan Africa,1136052685,11360.52685,7796609105,77966.09105


In [67]:
country_populations = country_populations.merge(region_populations,how="left",on="Region")
country_populations.head()

,Level,Region,Country,State/Province,Population,Population 100K,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,Region Population,Region Population 100K,World Population,World Population 100K
0,Country,Central Asia,Armenia,,2963243,29.63243,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,326887719,3268.87719,7796609105,77966.09105
1,Country,Central Asia,Azerbaijan,,10139177,101.39177,0.13,56.0,0.91,91459.0,1200.0,123.0,82658.0,2.1,32.0,326887719,3268.87719,7796609105,77966.09105
2,Country,Central Asia,Cyprus,,1207359,12.07359,0.02,67.0,0.73,8784.0,5000.0,131.0,9240.0,1.3,37.0,326887719,3268.87719,7796609105,77966.09105
3,Country,Central Asia,Faeroe Islands,,48863,0.48863,0.00,43.0,0.38,185.0,NaN,35.0,1396.0,NaN,NaN,326887719,3268.87719,7796609105,77966.09105
4,Country,Central Asia,Georgia,,3989167,39.89167,0.05,58.0,-0.19,-7598.0,-10000.0,57.0,69490.0,2.1,38.0,326887719,3268.87719,7796609105,77966.09105


In [68]:
us_populations = pd.read_excel(us_population_file)
us_populations = us_populations.drop(["Population Source"],axis=1)
us_populations["Region"] = "North America"
us_populations["Level"] = us_populations["State/Province"].apply(lambda x: "Country" if x=="United States" else "State/Province")
us_populations["State/Province"] = us_populations["State/Province"].apply(lambda x: "" if x=="United States" else x)
us_populations.head(75)

us_country_population = us_populations.loc[us_populations["Census Region"]=="United States"].copy().reset_index()
us_country_population = us_country_population[["Country","Population","Population 100K"]] 
us_country_population["Level"] = "State/Province"
us_country_population.rename(columns={
    "Population":"Country Population",
    "Population 100K":"Country Population 100K"
},inplace=True)
us_country_population.head()

us_populations = us_populations.merge(us_country_population,how="left",on=["Level","Country"])
us_populations.head()

,Country,State/Province,Census Region,Population,Population 100K,Female,Male,Pct Male,Pct Female,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+,Region,Level,Country Population,Country Population 100K
0,United States,Alabama,South,4889347,48.89347,2531653,2357694,0.482210,0.517790,56901,58290,59073,59799,60294,59568,58599,59537,60023,60241,60897,63083,62906,61883,61729,61740,61799,61924,62938,64125,63587,64201,63943,63719,63922,65079,65208,67027,69478,68758,64852,61469,59980,59615,60721,58941,59921,60346,60696,62200,58159,57993,57852,55498,58174,57008,58838,61959,65460,64750,60738,59494,59786,61321,65925,66906,66695,67073,67308,68221,65605,65211,65365,63117,62042,59584,56766,54694,52697,51707,50567,49884,51612,37091,36845,35441,36173,30575,27572,26053,23977,22580,19594,18222,16660,91543,237456,608466,631898,642187,589780,615279,657543,501447,256847,0.011638,0.048566,0.124447,0.129240,0.131344,0.120626,0.125841,0.134485,0.102559,0.052532,0.018723,North America,State/Province,327052602.0,3270.52602
1,United States,Alaska,West,712114,7.12114,347065,365049,0.512627,0.487373,9978,10012,10186,10509,10395,10414,10303,10286,10436,10157,9976,10016,9887,9509,9678,9488,9410,9343,8518,7525,8088,8617,9132,9252,9900,10318,10693,11456,11576,11552,10946,10809,10460,10822,10799,10303,10452,9962,9667,9685,8865,8589,8533,7954,8295,7827,7962,8230,8696,9086,8428,8197,8330,8595,9128,9426,9493,9636,9731,9894,9373,9168,9161,8614,8800,8210,7734,7220,6655,6442,5978,5621,5482,4013,3945,3665,3459,2950,2677,2327,1971,1784,1586,1411,1277,7181,41102,100662,89273,109431,92305,84479,93296,61300,23107,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084,North America,State/Province,327052602.0,3270.52602
2,United States,Arizona,West,7259090,72.59090,3658425,3600665,0.496022,0.503978,81929,83065,85726,88192,90876,90858,90405,90319,91313,90797,92572,96605,96951,95849,95076,94742,91832,93119,97724,99334,95779,96720,97447,99492,100066,101835,102053,105016,106612,105885,99473,96012,93463,94419,94495,91292,93193,93126,92593,94337,87480,85369,85480,83038,86427,83705,82925,85342,90167,90646,85428,82772,81434,81904,87011,88220,88617,88864,90069,90909,88041,87973,87664,85106,85274,83374,81125,78896,77728,77376,76474,76787,81299,59741,58899,57521,58174,50088,45100,41784,38116,35128,31252,28331,25703,145737,347859,930745,966255,999263,892335,851334,880737,751699,411197,0.011286,0.047920,0.128218,0.133110,0.137657,0.122927,0.117278,0.121329,0.103553,0.056646,0.020076,North America,State/Province,327052602.0,3270.52602
3,United States,Arkansas,South,3012542,30.12542,1535409,1477133,0.490328,0.509672,36355,37006,37572,38610,38921,38404,37924,38827,38633,38959,38941,40404,41015,40146,39960,39598,39485,39395,38933,39714,40206,40211,40323,39367,38992,39539,39518,40912,42271,41927,39361,38289,37446,37354,37897,37550,38010,38198,38328,39332,36427,36037,35410,34319,35486,34449,34938,36485,37966,38136,35481,34716,34766,35572,38680,39707,39698,39360,39525,39697,38668,38092,37865,36917,36430,35478,34176,32682,31736,30888,30427,30017,31554,22864,23007,22169,22217,19362,17669,16670,14936,13764,12330,11253,10771,59912,152109,393213,396224,394514,369097,361189,385959,302829,161141,0.012068,0.050492,0.130525,0.131525,0.130957,0.122520,0.119895,0.128117,0.100523,0.053490,0.019888,North America,State/Province,327052602.0,3270.52602
4,United States,California,West,39356141,393.56141,19843586,19512555,0.495794,0.504206,462589,462713,477322,485894,495198,493458,494221,493396,504330,493445,492283,511109,512662,507455,505628,503712,501846,497188,515261,501692,493088,497749,512251,533604,557011,576604

In [69]:
# Canada Population
population_cn = pd.read_excel(canada_population_file)
population_cn.rename(columns = {"GEO": "State/Province",
                                "VALUE": "Population",
                                "REF_DATE": "Quarter"}, inplace = True)
population_cn = population_cn[["Quarter","State/Province","Population"]]
last_quarter = population_cn["Quarter"].max()
print("Canada Populations " + last_quarter)
canada_last_population = population_cn.loc[population_cn["Quarter"]==last_quarter].copy()[["State/Province","Population"]]
canada_last_population["Population 100K"] = canada_last_population["Population"]/100000
canada_last_population.reset_index(drop=True,inplace=True)
canada_last_population["Country"] = "Canada"
canada_last_population["Level"] = canada_last_population["State/Province"].apply(lambda x: "Country" if x=="Canada" else "State/Province")
canada_last_population["State/Province"] = canada_last_population["State/Province"].apply(lambda x: "" if x=="Canada" else x)
canada_last_population["Region"] = "North America"
canada_order = ['Level','Region','Country','State/Province', 'Population', 'Population 100K']
canada_last_population = canada_last_population[canada_order]
canada_last_population.head(20)

cn_country_population = canada_last_population.loc[canada_last_population["State/Province"]==""].copy().reset_index()
cn_country_population = cn_country_population[["Country","Population","Population 100K"]]
cn_country_population["Level"] = "State/Province"
cn_country_population.rename(columns={
    "Population":"Country Population",
    "Population 100K":"Country Population 100K"
},inplace=True)
cn_country_population.head()

canada_last_population = canada_last_population.merge(cn_country_population,how="left",on=["Level","Country"])
canada_last_population.head(15)

Canada Populations 2020-07


,Level,Region,Country,State/Province,Population,Population 100K,Country Population,Country Population 100K
0,Country,North America,Canada,,38005238,380.05238,NaN,NaN
1,State/Province,North America,Canada,Newfoundland and Labrador,522103,5.22103,38005238.0,380.05238
2,State/Province,North America,Canada,Prince Edward Island,159625,1.59625,38005238.0,380.05238
3,State/Province,North America,Canada,Nova Scotia,979351,9.79351,38005238.0,380.05238
4,State/Province,North America,Canada,New Brunswick,781476,7.81476,38005238.0,380.05238
5,State/Province,North America,Canada,Quebec,8574571,85.74571,38005238.0,380.05238
6,State/Province,North America,Canada,Ontario,14734014,147.34014,38005238.0,380.05238
7,State/Province,North America,Canada,Manitoba,1379263,13.79263,38005238.0,380.05238
8,State/Province,North America,Canada,Saskatchewan,1178681,11.78681,38005238.0,380.05238
9,State/Province,North America,Canada,Alberta,4421876,44.21876,38005238.0,380.05238


In [70]:
na_country_populations = pd.concat([us_country_population,cn_country_population]).reset_index()
na_country_populations.drop(columns=["index","Country"],inplace=True)
na_country_populations.head()

na_region_population = na_country_populations.groupby(["Level"]).sum().reset_index()
na_region_population.rename(columns={
    "Country Population":"Region Population",
    "Country Population 100K":"Region Population 100K"
},inplace=True)
na_region_population.drop(columns=["Level"],inplace=True)
na_region_population["Region"] = "North America"
na_region_population.head()

,Region Population,Region Population 100K,Region
0,365057840,3650.5784,North America


In [71]:
na_populations = pd.concat([canada_last_population,us_populations],sort=True).reset_index()
na_populations.drop(columns=["index"],inplace=True)
na_populations = na_populations.merge(na_region_population,how="left",on="Region")
na_populations["World"] = "World"
na_populations = na_populations.merge(world_population,how="left",on=["World"])
na_order = [
    "Level","Region","Country","Census Region","State/Province",
    "Population","Population 100K","Country Population","Country Population 100K",
    "Region Population","Region Population 100K","World Population","World Population 100K",
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4','5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84',
    'Pct < 1','Pct 1-4','Pct 5-14','Pct 15-24','Pct 25-34','Pct 35-44','Pct 45-54','Pct 55-64','Pct 65-74','Pct 75-84','Pct 85+'
]
na_populations = na_populations[na_order]
na_populations.head(75)

,Level,Region,Country,Census Region,State/Province,Population,Population 100K,Country Population,Country Population 100K,Region Population,Region Population 100K,World Population,World Population 100K,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,Country,North America,Canada,NaN,,38005238,380.05238,NaN,NaN,365057840,3650.5784,7796609105,77966.09105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State/Province,North America,Canada,NaN,Newfoundland and Labrador,522103,5.22103,38005238.0,380.05238,365057840,3650.5784,7796609105,77966.09105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,State/Province,North America,Canada,NaN,Prince Edward Island,159625,1.59625,38005238.0,380.05238,365057840,3650.5784,7796609105,77966.09105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,State/Province,North America,Canada,NaN,Nova Scotia,979351,9.79351,38005238.0,380.05238,365057840,3650.5784,7796609105,77966.09105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,State/Province,North America,Canada,NaN,New Brunswick,781476,7.81476,38005238.0,380.05238,365057840,3650.5784,7796609105,77966.09105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,State/Province,North America,United States,South,Virginia,8420244,84.20244,327052602.0,3270.52602,365057840,3650.5784,7796609105

In [72]:
country_populations = country_populations[((country_populations["Country"] != "United States") & (country_populations["Country"] != "Canada"))]
print(country_populations.columns)
all_populations = pd.concat([na_populations,country_populations],sort=True).reset_index()
all_populations.drop(columns=["index"],inplace=True)
all_populations["Country Share"] = all_populations.apply(
    lambda x: x["Population"]/x["Country Population"] if x["Level"]=="State/Province" or x["Level"]=="Territory" else None,
    axis=1
)
all_populations["Country Share"] = all_populations["Country Share"].round(9)
all_populations["Region Share"] = all_populations["Population"]/all_populations["Region Population"]
all_populations["Region Share"] = all_populations["Region Share"].round(9)
all_populations["World Share"] = all_populations["Population"]/all_populations["World Population"]
all_populations["World Share"] = all_populations["World Share"].round(9)
all_order = [
    "Level","Region","Country","Census Region","State/Province",
    "Population","Population 100K","Country Population","Country Population 100K",
    "Region Population","Region Population 100K","World Population","World Population 100K",
    "Country Share","Region Share","World Share",
    'World Share (%)', 'Urban Population (%)', 'Annual Change (%)', 'Net Change', 'Migrants (net)', 'Density (P/Km²)',
    'Land Area (Km²)', 'Fertility Rate', 'Median Age',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4','5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84',
    'Pct < 1','Pct 1-4','Pct 5-14','Pct 15-24','Pct 25-34','Pct 35-44','Pct 45-54','Pct 55-64','Pct 65-74','Pct 75-84','Pct 85+'
]
all_populations = all_populations[all_order]
all_populations = all_populations.sort_values(by=["Region","Country","State/Province"]).reset_index().drop(columns=["index"])
all_populations.to_excel(cleanedFolder + "all_populations.xlsx", index=False)
all_populations.head()

Index(['Level', 'Region', 'Country', 'State/Province', 'Population',
       'Population 100K', 'World Share (%)', 'Urban Population (%)',
       'Annual Change (%)', 'Net Change', 'Migrants (net)', 'Density (P/Km²)',
       'Land Area (Km²)', 'Fertility Rate', 'Median Age', 'Region Population',
       'Region Population 100K', 'World Population', 'World Population 100K'],
      dtype='object')


,Level,Region,Country,Census Region,State/Province,Population,Population 100K,Country Population,Country Population 100K,Region Population,Region Population 100K,World Population,World Population 100K,Country Share,Region Share,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,Country,Central Asia,Armenia,NaN,,2963243,29.63243,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.009065,0.000380,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country,Central Asia,Azerbaijan,NaN,,10139177,101.39177,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.031017,0.001300,0.13,56.0,0.91,91459.0,1200.0,123.0,82658.0,2.1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country,Central Asia,Cyprus,NaN,,1207359,12.07359,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.003693,0.000155,0.02,67.0,0.73,8784.0,5000.0,131.0,9240.0,1.3,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country,Central Asia,Faeroe Islands,NaN,,48863,0.48863,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.000149,0.000006,0.00,43.0,0.38,185.0,NaN,35.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Country,Central Asia,Georgia,NaN,,3989167,39.89167,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.012203,0.000512,0.05,58.0,-0.19,-7598.0,-10000.0,57.0,69490.0,2.1,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
